In [51]:
from bs4 import BeautifulSoup
import requests

import pandas as pd
import random
import math
import time
import json
import re

In [52]:
# Load previous saved datafram
df = pd.read_csv('movie_info_df.csv')

In [ ]:
reviews_dict = {'id': [], 'reviews': [], 'rating': []}
errors = []
c = 0

for i in df['id'].unique():
    c += 1
    link = 'https://www.imdb.com/title/tt' + str(i) + '/reviews'
    
    try:
        html = requests.get(link, headers={'user-agent': 'Mozilla/5.0'}).content
        soup = BeautifulSoup(html, features="lxml")
        time.sleep(random.uniform(0.5,1))
        
        reviews = [i.get_text(' ', strip=True).replace('\n', ' ')
                   if soup.find_all('div', {'class':'text show-more__control'}) else None
                   for i in soup.find_all('div', {'class':'text show-more__control'})]
        
        rating = [i.find('span', {'class':'rating-other-user-rating'}).get_text(' ', strip=True)[0]
                    if i.find('span', {'class':'rating-other-user-rating'}) else None
                    for i in soup.find_all('div', {'class':'review-container'})]

        
        reviews_dict['id'] += [i]*len(reviews)
        reviews_dict['reviews'] += reviews
        reviews_dict['rating'] += rating
        
    except:
        errors += [link]
        
    # Saving after every 5000 links    
    if c%5000 == 0:
        print(c, 'links scraped.')
        time.sleep(1)
        print("Saving... don't quit!")
        
        with open('movie_reviews_raw_1.json', 'w') as outfile:
            json.dump(reviews_dict, outfile)
            
print('You got {} right and {} errors.'.format(len(reviews_dict), len(errors)))

In [ ]:
# Checking order and results

assert len(reviews_dict['id']) == len(reviews_dict['reviews']) == len(reviews_dict['rating'])

print("You have scraped {} reviews of {} movies.".format(len(reviews_dict['reviews']),
                                                         len(df['id'].unique())-len(errors)))

In [ ]:
# Turn it into a dataframe
df = pd.DataFrame.from_dict(reviews_dict)

df.head()

In [ ]:
# Saving to file
df.to_csv('reviews_df.csv')

In [ ]:
with open('movie_reviews_raw_1.json', 'r') as outfile:
           reviews_dict = json.load(outfile)

In [15]:
for k,v in reviews_dict.items():
    print(len(v))

24363
24363
24364


In [28]:
[type(i) for i in reviews_dict['rating'] if type(i) is not str and i is not None]

[]

In [49]:
reviews_dict['rating'][0]

'8'

In [50]:
reviews_dict['reviews'][0]

"After watching this movie on Lifetime I have to say that this movie was very good with just a few tiny hiccups. Jamie Luner plays Sandra who suspects that her husband, Matthew is cheating on her when she begins to receive warnings through the mail and email via the computer. She starts to follow him around town, seeing him purchase a set of pearls in a jewelry store and later that day or the next he gives her a ring from the same jewelry store she decides to hire a detective couple to follow him and gain evidence of the affair. What they soon learn is that her husband is not cheating on her and she tells her husband about hiring the detectives. The detectives then start to track down the helpful citizen that sent the emails and letters through the mail. Sandra then has her hands full with an art show debuting her new works of art. The story culminates in Sandra learning that Matthew's partner, Bill has been supposedly stalking her and wants her all for himself. Small inconsistencies i

In [47]:
reviews_dict['id'][0]

1295093